In [3]:
from seaborn import load_dataset

In [5]:
tips = load_dataset('tips')

In [7]:
tips['tip_pct'] = tips['tip']/tips['total_bill']

aggregate와 마찬가지로 `transform`은 엄격한 요구사항을 갖는 특수한 목적의 함수다.<br/>
인자로 넘긴 함수는 반드시 스칼라 값을 생성해야 하며(np.mean 처럼) 혹은 같은 크기를 가지는 변형된 배열을 생성해야 한다.<br/>
가장 일반적인 GroupBy 메서드의 목적은 `apply`인데, 이는 지금부터 다루게 될 주제이다.<br/>
`apply` 메서드는 객체를 여러 조각으로 나누어 전달된 함수를 각 조각에 일괄적으로 적용한 후 이를 다시 합치게 된다.

tips 데이터 에서 그룹별 상위 5개의 tip_pct값을 골라보자.<br/>
우선 특정 컬럼에서 가장 큰 값을 가지는 로우를 선택하는 함수를 바로 작성해보자.

In [1]:
def top(df, n=5, columns='tip_pct'):
    return df.sort_values(by=columns)[-n:]

In [13]:
top(tips, n=1)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [8]:
top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [9]:
tips.groupby('smoker').count()

,total_bill,tip,sex,day,time,size,tip_pct
smoker,,,,,,,
Yes,93,93,93,93,93,93,93
No,151,151,151,151,151,151,151


In [10]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990

만일 `apply` 메서드에 넘길 함수가 추가적인 인자를 받는다면 함수 이름 뒤에 붙여서 넘겨주면 된다.

In [15]:
tips.groupby(['smoker','day']).apply(top, n=1, columns='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
Yes    Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   
       Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
No     Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
       Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   

                  tip_pct  
smoker day                 
Yes    Thur 197  0.115982  
       Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
No     Thur 142  0.121389  
       Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799

### 그룹 색인 생략하기
앞에서 살펴본 예제에서 반환된 객체들은 원본 객체의 각 조각에 대한 색인과 그룹 키가 계층적 색인으로 사용되는 걸 볼 수 있었다.<br/>
이런 결과는 groupby 메서드에 `group_keys=False`를 넘겨서 막을 수 있다.

In [17]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990


### 색인되지 않은 형태로 집계된 데이터 반환하기
`as_index=False` 사용

In [21]:
tips.groupby('smoker',as_index=False).apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
0 109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
  183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
  67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
  178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
  172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345
1 88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
  185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
  51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
  149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
  232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990